In [1]:
import os
import glob
import numpy as np
import scipy
import astropy.table
import astropy.io.fits
import astropy.units
import random
import copy

import matplotlib.pyplot as plt

import lezargus


%matplotlib Qt

ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty

In [2]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/spectre_calibrations/", exist_ok=True)

In [3]:
def read_dat_arc_files(filename: str) -> tuple[list, list]:
    """Read in a text/dat based arc data file."""
    loaded_data = np.genfromtxt(filename, dtype=float, comments="#")
    raw_wavelength, raw_flux = loaded_data.T

    # We want to ensure that no NaNs screw things up so we just interpolate
    # in the middle in NaN cases.
    wavelength = raw_wavelength
    interpolator = lezargus.library.interpolate.Linear1DInterpolate(
        x=raw_wavelength, v=raw_flux, extrapolate=False
    )
    flux = interpolator(wavelength)

    return wavelength, flux


def read_spex_arc_files(filename: str) -> tuple[list, list]:
    """Read in the SpeX arc data."""
    with astropy.io.fits.open(filename) as hdul:
        hdu = hdul["PRIMARY"]
        hdu_data = hdu.data
        raw_wavelength, raw_flux, raw_error, __ = hdu_data

    # We want to ensure that no NaNs screw things up so we just interpolate
    # in the middle in NaN cases.
    wavelength = raw_wavelength
    interpolator = lezargus.library.interpolate.Linear1DInterpolate(
        x=raw_wavelength, v=raw_flux, extrapolate=False
    )
    flux = interpolator(wavelength)

    # All done.
    return wavelength, flux

## Simulation Th Ar Lamps

In [4]:
thar_lamp_filename = "./base/spectre_calibrations/thar_spec_MM201006_r2000.dat"
thar_wave, thar_flux = read_dat_arc_files(filename=thar_lamp_filename)

# Converting the wavelength and data.
thar_wave_si = lezargus.library.conversion.convert_units(
    value=thar_wave, value_unit="um", result_unit="m"
)
thar_flux_si = lezargus.library.conversion.convert_units(
    value=thar_flux, value_unit="count s^-1", result_unit="count s^-1"
)

thar_wave_column = astropy.table.Column(
    thar_wave_si, name="wavelength", unit="m", dtype=float
)
thar_flux_column = astropy.table.Column(
    thar_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
thar_table_columns = [thar_wave_column, thar_flux_column]
thar_arclamp_table = astropy.table.Table(thar_table_columns)
# Too much precision makes unnessary large file sizes.
thar_out_filename = f"./products/spectre_calibrations/visible_thar_arclamp.dat"
thar_arclamp_table.write(
    thar_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in thar_arclamp_table.keys()},
    overwrite=True,
)

## Simulation Hg Ar Arcs

In [5]:
hgar_lamp_filename = (
    "./base/spectre_calibrations/hgar_arcspectrum_0p4_1p0um.dat"
)
hgar_wave, hgar_flux = read_dat_arc_files(filename=hgar_lamp_filename)

# Converting the wavelength and data.
hgar_wave_si = lezargus.library.conversion.convert_units(
    value=hgar_wave, value_unit="um", result_unit="m"
)
hgar_flux_si = lezargus.library.conversion.convert_units(
    value=hgar_flux, value_unit="count s^-1", result_unit="count s^-1"
)

# Small delta to match up with SXD/LXDs level arc lamp levels.
hgar_flux_si = hgar_flux_si * 1 * 1e4

hgar_wave_column = astropy.table.Column(
    hgar_wave_si, name="wavelength", unit="m", dtype=float
)
hgar_flux_column = astropy.table.Column(
    hgar_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
hgar_table_columns = [hgar_wave_column, hgar_flux_column]
hgar_arclamp_table = astropy.table.Table(hgar_table_columns)
# Too much precision makes unnessary large file sizes.
hgar_out_filename = f"./products/spectre_calibrations/visible_hgar_arclamp.dat"
hgar_arclamp_table.write(
    hgar_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in hgar_arclamp_table.keys()},
    overwrite=True,
)

## Simulation SXD Arcs

In [6]:
spex_sxd_filename = "./base/spectre_calibrations/spex_sxd_0p3slit_arc.fits"
sxd_wave, sxd_flux = read_spex_arc_files(filename=spex_sxd_filename)

# Converting the wavelength and data.
sxd_wave_si = lezargus.library.conversion.convert_units(
    value=sxd_wave, value_unit="um", result_unit="m"
)
sxd_flux_si = lezargus.library.conversion.convert_units(
    value=sxd_flux, value_unit="count s^-1", result_unit="count s^-1"
)

sxd_wave_column = astropy.table.Column(
    sxd_wave_si, name="wavelength", unit="m", dtype=float
)
sxd_flux_column = astropy.table.Column(
    sxd_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
sxd_table_columns = [sxd_wave_column, sxd_flux_column]
sxd_arclamp_table = astropy.table.Table(sxd_table_columns)
# Too much precision makes unnessary large file sizes.
sxd_out_filename = (
    f"./products/spectre_calibrations/nearir_spex_sxd_arclamp.dat"
)
sxd_arclamp_table.write(
    sxd_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in sxd_arclamp_table.keys()},
    overwrite=True,
)

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


## Simulation LXDS Arcs

In [7]:
spex_lxds_filename = (
    "./base/spectre_calibrations/spex_lxdshort_0p8slit_arc.fits"
)
lxds_wave, lxds_flux = read_spex_arc_files(filename=spex_lxds_filename)

# Converting the wavelength and data.
lxds_wave_si = lezargus.library.conversion.convert_units(
    value=lxds_wave, value_unit="um", result_unit="m"
)
lxds_flux_si = lezargus.library.conversion.convert_units(
    value=lxds_flux, value_unit="count s^-1", result_unit="count s^-1"
)

lxds_wave_column = astropy.table.Column(
    lxds_wave_si, name="wavelength", unit="m", dtype=float
)
lxds_flux_column = astropy.table.Column(
    lxds_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
lxds_table_columns = [lxds_wave_column, lxds_flux_column]
lxds_arclamp_table = astropy.table.Table(lxds_table_columns)
# Too much precision makes unnessary large file sizes.
lxds_out_filename = (
    f"./products/spectre_calibrations/midir_spex_lxds_arclamp.dat"
)
lxds_arclamp_table.write(
    lxds_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in lxds_arclamp_table.keys()},
    overwrite=True,
)

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


## Simulation SPECTRE Wavelength Solution

## Visible Channel

In [8]:
visible_simulate = lezargus.simulator.SpectreSimulator.from_blackbody(
    wavelength=np.linspace(0.4, 1.00, 2000) * 1e-6,
    blackbody_temperature=3000,
    magnitude=10,
    photometric_filter=lezargus.data.FILTER_JOHNSON_V,
    spectral_scale=3.1e-8,
    channel="visible",
    exposure_time=60.0,
    spatial_oversample=2,
    zenith_angle=np.deg2rad(60),
    parallactic_angle=np.deg2rad(45),
)
# Arc lamp in.
visible_simulate.arc_lamp_in = True
visible_arclamp_image = copy.deepcopy(visible_simulate.at_scattered_light)
# And the flat lamp in.
visible_simulate.clear_cache()
visible_simulate.arc_lamp_in = False
visible_simulate.flat_lamp_in = True
visible_flatlamp_image = copy.deepcopy(visible_simulate.at_scattered_light)

# Noise adaption...
visible_flatlamp_image.data += (
    np.random.random(size=visible_flatlamp_image.data.shape) / 1e6
)
visible_arclamp_image.data += (
    np.random.random(size=visible_arclamp_image.data.shape) / 1e6
)

# The retriever...
visible_retriever = lezargus.pipeline.SpectreRetrieval(
    flat_image=visible_flatlamp_image,
    arc_image=visible_arclamp_image,
    channel="visible",
    image=None,
)

[Lezargus] 2026-02-12T19:51:32    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:51:32    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T19:51:32    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T19:51:34    ERROR -- ToDoError - Simulated arc lamps are broken into 3 channels; but there is only one arc lamp. A full spectrum one should be used.
[Lezargus] 2026-02-12T19:51:34  WARNING -- InputWarning - The contain array fully exceeds the base array, which is not the intention of the inputs. The inputs may need to be reversed.
[Lezargus] 2026-02-12T19:51:34    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:51:48    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T1

In [9]:
slice_index = 15

# Extract the wavelength slice...
wavelength_slice = visible_retriever.retrieve_slice(
    image=visible_arclamp_image,
    slice_=slice_index,
    buffer_width=7,
    rotate=False,
)
wavelength_slice_array = wavelength_slice.data
# And create the arclamp spectrum, though this is an array...
arclamp_spectrum = np.mean(wavelength_slice_array, axis=1)

# This is needed for the next step, determining the wavelength solution truth
# manually...
try:
    visible_simulation_arclamp = (
        lezargus.data.SPECTRE_ARCLAMP_SIMULATION_VISIBLE
    )
except AttributeError:
    # It likely does not exist, and if we are running this as a script, it
    # does not really matter. So, no need.
    visible_simulation_arclamp = None
    visible_simulation_arclamp_convolved = None
else:
    smoothing_kernel = lezargus.library.convolution.kernel_1d_gaussian(100, 7)
    visible_simulation_arclamp_convolved = visible_simulation_arclamp.convolve(
        kernel=smoothing_kernel
    )

[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T19:57:23    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.


In [10]:
if visible_simulation_arclamp is not None:
    plt.figure()
    plt.title("Visible Arclamp")
    plt.plot(
        visible_simulation_arclamp.wavelength, visible_simulation_arclamp.data
    )
    plt.show()
if visible_simulation_arclamp_convolved is not None:
    plt.figure()
    plt.title("Visible Arclamp Convolved")
    plt.plot(
        visible_simulation_arclamp_convolved.wavelength,
        visible_simulation_arclamp_convolved.data,
    )
    plt.show()

plt.figure()
plt.title("Simulation Data")
plt.plot(arclamp_spectrum)
plt.show()

# This translation table was created from manually reading off the plots
visible_translation_table = {
    543: 0.404,
    534: 0.436,
    458: 0.491,
    273: 0.546,
    236: 0.578,
    180: 0.640,
    170: 0.653,
    165: 0.660,
    141: 0.696,
    135: 0.707,
    130: 0.715,
    126: 0.720,
    123: 0.727,
    116: 0.738,
    109: 0.750,
    103: 0.764,
    98: 0.772,
    87: 0.795,
    84: 0.801,
    80: 0.811,
    73: 0.826,
    66: 0.842,
    62: 0.852,
    56: 0.866,
    38: 0.912,
    34: 0.922,
    19: 0.965,
    14: 0.978,
}

# Interpolating out the table.
translation_pixels = np.array(
    list(visible_translation_table.keys()), dtype=float
)
translation_wavelength = np.array(
    list(visible_translation_table.values()), dtype=float
)
translation_wavelength_si = translation_wavelength * 1e-6
arclamp_wavelength_function = lezargus.library.interpolate.Linear1DInterpolate(
    x=translation_pixels, v=translation_wavelength_si, extrapolate=True
)
arclamp_wavelength = arclamp_wavelength_function(
    np.arange(arclamp_spectrum.size)
)

# And saving it as a file.
wavelength_column = astropy.table.Column(
    arclamp_wavelength, name="wavelength", unit="m", dtype=float
)
flux_column = astropy.table.Column(
    arclamp_spectrum, name="flux", unit="count s^-1", dtype=float
)
# Readout.
solution_table_columns = [wavelength_column, flux_column]
solution_table = astropy.table.Table(solution_table_columns)
# Too much precision makes unnessary large file sizes.
visible_solution_filename = (
    f"./products/spectre_calibrations/visible_wavelength_solution.dat"
)
solution_table.write(
    visible_solution_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in solution_table.keys()},
    overwrite=True,
)

In [11]:
plt.figure()
for wavedex in translation_wavelength_si:
    plt.axvline(wavedex, color="g", alpha=0.5)
plt.plot(arclamp_wavelength, arclamp_spectrum)

plt.show()

## Near IR Channel

In [12]:
nearir_simulate = lezargus.simulator.SpectreSimulator.from_blackbody(
    wavelength=np.linspace(0.70, 2.55, 2000) * 1e-6,
    blackbody_temperature=3000,
    magnitude=10,
    photometric_filter=lezargus.data.FILTER_JOHNSON_V,
    spectral_scale=3.1e-8,
    channel="nearir",
    exposure_time=60.0,
    spatial_oversample=2,
    zenith_angle=np.deg2rad(60),
    parallactic_angle=np.deg2rad(45),
)
# Arc lamp in.
nearir_simulate.arc_lamp_in = True
nearir_arclamp_image = copy.deepcopy(nearir_simulate.at_scattered_light)
# And the flat lamp in.
nearir_simulate.clear_cache()
nearir_simulate.arc_lamp_in = False
nearir_simulate.flat_lamp_in = True
nearir_flatlamp_image = copy.deepcopy(nearir_simulate.at_scattered_light)

# Noise adaption...
nearir_flatlamp_image.data += (
    np.random.random(size=nearir_flatlamp_image.data.shape) / 1e6
)
nearir_arclamp_image.data += (
    np.random.random(size=nearir_arclamp_image.data.shape) / 1e6
)

# The retriever...
nearir_retriever = lezargus.pipeline.SpectreRetrieval(
    flat_image=nearir_flatlamp_image,
    arc_image=nearir_arclamp_image,
    channel="nearir",
    image=None,
)

[Lezargus] 2026-02-12T19:57:29    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:57:29    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T19:57:29    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T19:57:30    ERROR -- ToDoError - Simulated arc lamps are broken into 3 channels; but there is only one arc lamp. A full spectrum one should be used.
[Lezargus] 2026-02-12T19:57:30    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:57:34    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:57:48    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T19:58:04    ERROR -- ToDoError - SPECTRE Window g

In [13]:
slice_index = 17

# Extract the wavelength slice...
wavelength_slice = nearir_retriever.retrieve_slice(
    image=nearir_arclamp_image,
    slice_=slice_index,
    buffer_width=7,
    rotate=False,
)
wavelength_slice_array = wavelength_slice.data
# And create the arclamp spectrum, though this is an array...
arclamp_spectrum = np.mean(wavelength_slice_array, axis=1)

# This is needed for the next step, determining the wavelength solution truth
# manually...
try:
    nearir_simulation_arclamp = lezargus.data.SPECTRE_ARCLAMP_SIMULATION_NEARIR
except AttributeError:
    # It likely does not exist, and if we are running this as a script, it
    # does not really matter. So, no need.
    nearir_simulation_arclamp = None
    nearir_simulation_arclamp_convolved = None
else:
    smoothing_kernel = lezargus.library.convolution.kernel_1d_gaussian(100, 5)
    nearir_simulation_arclamp_convolved = nearir_simulation_arclamp.convolve(
        kernel=smoothing_kernel
    )

[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:01:12    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.


In [14]:
if nearir_simulation_arclamp is not None:
    plt.figure()
    plt.title("Nearir Arclamp")
    plt.plot(
        nearir_simulation_arclamp.wavelength, nearir_simulation_arclamp.data
    )
    plt.show()
if nearir_simulation_arclamp_convolved is not None:
    plt.figure()
    plt.title("Nearir Arclamp Convolved")
    plt.plot(
        nearir_simulation_arclamp_convolved.wavelength,
        nearir_simulation_arclamp_convolved.data,
    )
    plt.show()
plt.figure()
plt.title("Simulation Data")
plt.plot(arclamp_spectrum)
plt.show()

# This translation table was created from manually reading off the plots
nearir_translation_table = {
    55: 2.397,
    58: 2.385,
    80: 2.314,
    109: 2.208,
    125: 2.154,
    130: 2.134,
    139: 2.099,
    150: 2.062,
    157: 2.032,
    167: 1.997,
    171: 1.984,
    207: 1.843,
    220: 1.792,
    231: 1.745,
    244: 1.697,
    318: 1.410,
    329: 1.371,
    334: 1.351,
    341: 1.328,
    351: 1.296,
    366: 1.245,
    392: 1.167,
    399: 1.149,
    429: 1.067,
    437: 1.047,
    469: 0.979,
    476: 0.966,
    500: 0.922,
    505: 0.913,
    546: 0.852,
    554: 0.842,
    567: 0.827,
    580: 0.811,
    589: 0.801,
    596: 0.795,
    618: 0.773,
    627: 0.764,
    639: 0.751,
    652: 0.738,
}

# Interpolating out the table.
translation_pixels = np.array(
    list(nearir_translation_table.keys()), dtype=float
)
translation_wavelength = np.array(
    list(nearir_translation_table.values()), dtype=float
)
translation_wavelength_si = translation_wavelength * 1e-6
arclamp_wavelength_function = lezargus.library.interpolate.Linear1DInterpolate(
    x=translation_pixels, v=translation_wavelength_si, extrapolate=True
)
arclamp_wavelength = arclamp_wavelength_function(
    np.arange(arclamp_spectrum.size)
)

# And saving it as a file.
wavelength_column = astropy.table.Column(
    arclamp_wavelength, name="wavelength", unit="m", dtype=float
)
flux_column = astropy.table.Column(
    arclamp_spectrum, name="flux", unit="count s^-1", dtype=float
)
# Readout.
solution_table_columns = [wavelength_column, flux_column]
solution_table = astropy.table.Table(solution_table_columns)
# Too much precision makes unnessary large file sizes.
nearir_solution_filename = (
    f"./products/spectre_calibrations/nearir_wavelength_solution.dat"
)
solution_table.write(
    nearir_solution_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in solution_table.keys()},
    overwrite=True,
)

In [15]:
plt.figure()
for wavedex in translation_wavelength_si:
    plt.axvline(wavedex, color="g", alpha=0.5)
plt.plot(arclamp_wavelength, arclamp_spectrum)

plt.show()

## Mid IR Channel

In [16]:
midir_simulate = lezargus.simulator.SpectreSimulator.from_blackbody(
    wavelength=np.linspace(1.65, 4.2, 2000) * 1e-6,
    blackbody_temperature=3000,
    magnitude=10,
    photometric_filter=lezargus.data.FILTER_JOHNSON_V,
    spectral_scale=3.1e-8,
    channel="midir",
    exposure_time=60.0,
    spatial_oversample=2,
    zenith_angle=np.deg2rad(60),
    parallactic_angle=np.deg2rad(45),
)
# Arc lamp in.
midir_simulate.arc_lamp_in = True
midir_arclamp_image = copy.deepcopy(midir_simulate.at_scattered_light)
# And the flat lamp in.
midir_simulate.clear_cache()
midir_simulate.arc_lamp_in = False
midir_simulate.flat_lamp_in = True
midir_flatlamp_image = copy.deepcopy(midir_simulate.at_scattered_light)

# Noise adaption...
midir_flatlamp_image.data += (
    np.random.random(size=midir_flatlamp_image.data.shape) / 1e6
)
midir_arclamp_image.data += (
    np.random.random(size=midir_arclamp_image.data.shape) / 1e6
)

# The retriever...
midir_retriever = lezargus.pipeline.SpectreRetrieval(
    flat_image=midir_flatlamp_image,
    arc_image=midir_arclamp_image,
    channel="midir",
    image=None,
)

[Lezargus] 2026-02-12T20:01:17    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T20:01:17    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T20:01:17    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T20:01:18    ERROR -- ToDoError - Simulated arc lamps are broken into 3 channels; but there is only one arc lamp. A full spectrum one should be used.
[Lezargus] 2026-02-12T20:01:18  WARNING -- AccuracyWarning - Interpolation is attempted at a wavelength beyond the domain of wavelengths of this spectrum. The overlap fraction is 0.9877045901580355.
[Lezargus] 2026-02-12T20:01:18    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T20:01:21    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargu

In [17]:
slice_index = 17

# Extract the wavelength slice...
wavelength_slice = midir_retriever.retrieve_slice(
    image=midir_arclamp_image,
    slice_=slice_index,
    buffer_width=7,
    rotate=False,
)
wavelength_slice_array = wavelength_slice.data
# And create the arclamp spectrum, though this is an array...
arclamp_spectrum = np.mean(wavelength_slice_array, axis=1)

# This is needed for the next step, determining the wavelength solution truth
# manually...
try:
    midir_simulation_arclamp = lezargus.data.SPECTRE_ARCLAMP_SIMULATION_MIDIR
except AttributeError:
    # It likely does not exist, and if we are running this as a script, it
    # does not really matter. So, no need.
    midir_simulation_arclamp = None
    midir_simulation_arclamp_convolved = None
else:
    smoothing_kernel = lezargus.library.convolution.kernel_1d_gaussian(100, 5)
    midir_simulation_arclamp_convolved = midir_simulation_arclamp.convolve(
        kernel=smoothing_kernel
    )

[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T20:04:51    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.


In [18]:
if midir_simulation_arclamp is not None:
    plt.figure()
    plt.title("Mid IR Arclamp")
    plt.plot(midir_simulation_arclamp.wavelength, midir_simulation_arclamp.data)
    plt.show()
if midir_simulation_arclamp_convolved is not None:
    plt.figure()
    plt.title("Mid IR Arclamp Convolved")
    plt.plot(
        midir_simulation_arclamp_convolved.wavelength,
        midir_simulation_arclamp_convolved.data,
    )
    plt.show()
plt.figure()
plt.title("Simulation Data")
plt.plot(arclamp_spectrum)
plt.show()

# This translation table was created from manually reading off the plots
midir_translation_table = {
    46: 4.088,
    61: 4.044,
    70: 4.026,
    72: 4.008,
    78: 3.991,
    85: 3.967,
    109: 3.891,
    136: 3.811,
    178: 3.691,
    187: 3.649,
    189: 3.633,
    193: 3.621,
    274: 3.329,
    279: 3.311,
    324: 3.133,
    360: 2.979,
    384: 2.878,
    387: 2.862,
    390: 2.850,
    424: 2.691,
    449: 2.567,
    452: 2.551,
    460: 2.513,
    481: 2.397,
    484: 2.385,
    497: 2.314,
    525: 2.154,
    528: 2.139,
    534: 2.099,
    539: 2.062,
    550: 1.997,
    552: 1.982,
    572: 1.843,
    576: 1.815,
    579: 1.792,
    586: 1.745,
}

# Interpolating out the table.
translation_pixels = np.array(list(midir_translation_table.keys()), dtype=float)
translation_wavelength = np.array(
    list(midir_translation_table.values()), dtype=float
)
translation_wavelength_si = translation_wavelength * 1e-6
arclamp_wavelength_function = lezargus.library.interpolate.Linear1DInterpolate(
    x=translation_pixels, v=translation_wavelength_si, extrapolate=True
)
arclamp_wavelength = arclamp_wavelength_function(
    np.arange(arclamp_spectrum.size)
)

# And saving it as a file.
wavelength_column = astropy.table.Column(
    arclamp_wavelength, name="wavelength", unit="m", dtype=float
)
flux_column = astropy.table.Column(
    arclamp_spectrum, name="flux", unit="count s^-1", dtype=float
)
# Readout.
solution_table_columns = [wavelength_column, flux_column]
solution_table = astropy.table.Table(solution_table_columns)
# Too much precision makes unnessary large file sizes.
midir_solution_filename = (
    f"./products/spectre_calibrations/midir_wavelength_solution.dat"
)
solution_table.write(
    midir_solution_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in solution_table.keys()},
    overwrite=True,
)

In [19]:
plt.figure()
for wavedex in translation_wavelength_si:
    plt.axvline(wavedex, color="g", alpha=0.5)
plt.plot(arclamp_wavelength, arclamp_spectrum)

plt.show()